In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import copy
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

from method.models import generate_random_values, generate_static_values, generate_llm_values
from method.ours.utils import create_driver, get_xpath, interact_with_input
from method.ours.history import HistoryTable
from method.ours.feedback import get_global_feedback

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

HEADLESS = False
URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
driver = create_driver(HEADLESS)
driver.get(URL)

In [5]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = driver.find_elements(By.TAG_NAME, 'form')[1]
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [ ]:
history_table = HistoryTable(
    url=URL,
    xpath=form_xpath
)

In [6]:
inputs

[<selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_11")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_13")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b97", element="86DC4A75B5B02B10C6306787E8ACE617_element_15")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2d35d17414249d2f5a8385b01a520b9

In [7]:
values = generate_llm_values(form.get_attribute('outerHTML'), openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate testing values for a web form. When given a web form, you need to generate values that can pass and fail when tried on the form. Your response should be in the following format:\n{\n[field_identifier]: {\npassing: <a base passing value for the field>,\nfailing: <a list of failing values for the field>\n}\n}\nYour response must be parsable by Python's json.loads function. Use \"id\" attribute to refer to the fields.\n"
  },
  {
    "role": "user",
    "content": "<form novalidate=\"\" autocomplete=\"off\" class=\"bkmg-tab-content-container bkmg-flights-tab-container ng-dirty ng-invalid ng-touched\"><fieldset><legend class=\"visually-hidden\"> Flights </legend><div class=\"bkmg-flights-tab-grid\"><fieldset class=\"bkmg-flights-tab-trip-type-selector mobile-padding\"><legend class=\"visually-hidden\"> Trip type </legend><abc-radio-group formcontrolname=\"tripType\" class=\"ng-untouched ng-pristine ng-valid ab

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n\"bkmgFlights_tripTypeSelector_R\": {\n\"passing\": \"R\",\n\"failing\": [\"O\", \"M\", \"A\", \"B\", \"C\", \"1\", \"2\", \"3\", \"\", null]\n},\n\"bkmgFlights_tripTypeSelector_O\": {\n\"passing\": \"O\",\n\"failing\": [\"R\", \"M\", \"A\", \"B\", \"C\", \"1\", \"2\", \"3\", \"\", null]\n},\n\"bkmgFlights_tripTypeSelector_M\": {\n\"passing\": \"M\",\n\"failing\": [\"O\", \"R\", \"A\", \"B\", \"C\", \"1\", \"2\", \"3\", \"\", null]\n},\n\"bkmgFlights_searchTypeToggle\": {\n\"passing\": true,\n\"failing\": [false, \"true\", \"false\", \"1\", \"0\", \"\", null]\n},\n\"bkmgFlights_origin_trip_1\": {\n\"passing\": \"Vancouver\",\n\"failing\": [\"\", null, \"123\", \"Vanc\", \"VancouverVancouverVancouverVancouverVancouverVancouver\"]\n},\n\"bkmgFlights_destination_trip_1\": {\n\"passing\": \"Toronto\",\n\"failing\": [\"\", null, \"123\", \"Toron\", \"TorontoTorontoTo

In [8]:
values

{'bkmgFlights_tripTypeSelector_R': {'passing': 'R',
  'failing': ['O', 'M', 'A', 'B', 'C', '1', '2', '3', '', None]},
 'bkmgFlights_tripTypeSelector_O': {'passing': 'O',
  'failing': ['R', 'M', 'A', 'B', 'C', '1', '2', '3', '', None]},
 'bkmgFlights_tripTypeSelector_M': {'passing': 'M',
  'failing': ['O', 'R', 'A', 'B', 'C', '1', '2', '3', '', None]},
 'bkmgFlights_searchTypeToggle': {'passing': True,
  'failing': [False, 'true', 'false', '1', '0', '', None]},
 'bkmgFlights_origin_trip_1': {'passing': 'Vancouver',
  'failing': ['',
   None,
   '123',
   'Vanc',
   'VancouverVancouverVancouverVancouverVancouverVancouver']},
 'bkmgFlights_destination_trip_1': {'passing': 'Toronto',
  'failing': ['',
   None,
   '123',
   'Toron',
   'TorontoTorontoTorontoTorontoTorontoToronto']},
 'bkmgFlights_travelDates_1-formfield-1': {'passing': '01/01',
  'failing': ['', None, '123', '01/1', '01/01/2022', '32/01', '01/13']},
 'bkmgFlights_travelDates_1-formfield-2': {'passing': '01/02',
  'failing':

In [20]:
def send_values(elements, values):
    for element_id in elements.keys():
        element = elements[element_id]
        value = values[element_id]
        
        element_type = element.get_attribute('type') or 'text'
    
        if element_type in ['radio', 'checkbox'] or element.tag_name in ['button']:
            continue
        
        interact_with_input(element, value)

        
def submit_form(form):
    submit = form.find_element(By.XPATH, f'{form_xpath}//*[@type="submit"]')
    
    for _ in range(3):
        try:
            interact_with_input(submit, True)
            time.sleep(0.5)
        except:
            break

In [21]:
passing_set = {
    element_id: values[element_id]['passing'] for element_id in values.keys()
}

elements = {
    element_id: driver.find_element(By.ID, element_id) for element_id in values.keys()
}


for element_id, element_values in values.items():
    element = elements[element_id]
    element_type = element.get_attribute('type') or 'text'
    
    if element_type in ['radio', 'checkbox']:
        continue
    
    for failing_value in element_values['failing']:
        passing_copy = copy.copy(passing_set)
        passing_copy[element_id] = failing_value
        send_values(elements, passing_copy)
        
        time.sleep(2)

        submit_form(form)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

        history_table.add(
            passing_copy,
            'base',
            global_feedback,
            driver.current_url
        )

NameError: name 'history_table' is not defined